In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# --- CORREÇÃO DOS IMPORTS (Para sumir os avisos amarelos) ---
import tensorflow as tf
import keras # Importa o Keras separadamente
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# Configurações Globais
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30 # Se estiver muito lento, mude para 10 ou 15
LEARNING_RATE = 0.001
base_dir = './dogs_vs_cats'


In [ ]:
import os
from PIL import Image

def limpar_imagens_corrompidas(folder_path):
    print(f"--- Verificando imagens em: {folder_path} ---")
    deleted_files = 0
    
    # Percorre todas as pastas e subpastas
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            
            try:
                # Tenta abrir a imagem
                with Image.open(file_path) as img:
                    img.verify() # Verifica a integridade do arquivo
            except (IOError, SyntaxError, Image.UnidentifiedImageError) as e:
                # Se der erro, deleta o arquivo
                print(f"Arquivo corrompido removido: {filename}")
                os.remove(file_path)
                deleted_files += 1

    if deleted_files == 0:
        print("Nenhuma imagem corrompida encontrada.")
    else:
        print(f"Total de arquivos removidos: {deleted_files}")

# Chama a função na sua pasta base
# Certifique-se que a variável 'base_dir' está definida como na Célula anterior
limpar_imagens_corrompidas(base_dir)

In [ ]:

print(f"Lendo imagens de: {os.path.abspath(base_dir)}")

# --- Configuração dos Geradores com Separação Automática ---

# Definimos o Data Augmentation E a divisão de validação (20%) aqui
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalização
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2         # <--- ISSO É NOVO: Separa 20% para teste automaticamente
)

# Gerador de Treino (usa 80% dos dados)
train_generator = train_datagen.flow_from_directory(
    base_dir,                    # Aponta para a pasta dogs_vs_cats
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'            # <--- Pega apenas a parte de treino
)

# Gerador de Teste/Validação (usa 20% dos dados)
test_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,               # Importante manter False para a Matriz de Confusão
    subset='validation'          # <--- Pega apenas a parte de validação
)

In [ ]:
# Célula 5: Definição da Arquitetura do Modelo

model = Sequential([
    # Primeiro Bloco Convolucional
    Conv2D(32, (3, 3), activation='relu', input_shape=IMG_SIZE + (3,)), # (150, 150, 3)
    MaxPooling2D(2, 2),

    # Segundo Bloco Convolucional
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Terceiro Bloco Convolucional
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Quarto Bloco Convolucional (Opcional, para mais complexidade)
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Camada Flatten para converter matrizes 2D em vetor 1D
    Flatten(),

    # Camada Densa (Fully Connected)
    Dense(512, activation='relu'),
    Dropout(0.5), # Desliga 50% dos neurônios aleatoriamente para evitar overfitting

    # Camada de Saída (BINÁRIA: 1 neurônio, sigmoid)
    Dense(1, activation='sigmoid')
])

# Compilação do Modelo
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy', # Obrigatório para classificação binária (0 ou 1)
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Célula 6: Treinamento com Callbacks

# Callbacks para melhorar o treinamento
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
]

print("Iniciando treinamento...")

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE, # Garante que passe por todas as imagens
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)

print("Treinamento concluído.")

In [ ]:
# Célula 7: Plotagem dos Gráficos de Desempenho

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

# Gráfico de Acurácia
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Acurácia de Treino')
plt.plot(epochs_range, val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.title('Acurácia de Treino e Validação')

# Gráfico de Perda (Loss)
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Perda de Treino')
plt.plot(epochs_range, val_loss, label='Perda de Validação')
plt.legend(loc='upper right')
plt.title('Perda de Treino e Validação')

plt.show()

In [ ]:
# Célula 8: Matriz de Confusão e Relatório de Classificação

# 1. Gerar previsões para o conjunto de teste/validação
print("Gerando previsões...")
predictions = model.predict(test_generator)

# 2. Converter probabilidades em classes (0 ou 1)
# Se probabilidade > 0.5 é Cachorro (1), senão é Gato (0)
y_pred = (predictions > 0.5).astype(int).ravel()

# 3. Pegar as classes reais (Ground Truth)
y_true = test_generator.classes

# 4. Pegar os nomes das classes (ex: 'cat', 'dog')
class_names = list(test_generator.class_indices.keys())

# --- Matriz de Confusão ---
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

# --- Relatório Completo ---
print("\nRelatório de Classificação:\n")
print(classification_report(y_true, y_pred, target_names=class_names))